In [1]:
import numpy as np
import pandas as pd

# from compiler.ast import flatten
from random import Random
from pandas import DataFrame
from numpy import log
from numpy import mat
from numpy import ones
from sklearn import metrics
from sklearn import tree
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import auc
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as pl

In [2]:
adult_header=["age","workclass","fnlwgt","education","education-num",
              "marital-status","occupation","relationship","race","sex","capital-gain",
              "capital-loss","hours-per-week","native-country","label"]
adult_data=pd.read_csv("./adult.data",index_col=False,names=adult_header)
adult_test=pd.read_csv("./adult2.test",index_col=False,names=adult_header)

In [3]:
adult_data[adult_data==" ?"]=np.nan
adult_test[adult_test==" ?"]=np.nan
#print(adult_data["age"])
adult_data.dropna(axis=0,how='any',inplace=True)
adult_test.dropna(axis=0,how='any',inplace=True)

/home/larryytr/anaconda3/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [4]:
discre_name=["workclass","education","marital-status",
             "occupation","relationship","race",
             "sex","native-country","label"]
for name in discre_name:
    key=np.unique(adult_data[name])
    #print(key)
    le=preprocessing.LabelEncoder()
    le.fit(key)
    adult_test[name]=le.transform(adult_test[name])
    adult_data[name]=le.transform(adult_data[name])
#print(adult_data)

In [5]:
X_data=adult_data.values[0:150,0:14]
#print(X_data[:,0:14])
Y_data=adult_data.values[0:150,14]
#print(Y_data)
X_test=adult_test.values[0:150,0:14]
#print(X_test[:,0:14])
Y_test=adult_test.values[0:150,14]
print(Y_test)
#记得删除

[0 0 1 1 0 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 1 0
 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0
 0 0 0 1 0 1 1 0 0 1 0 0 1 0 0 0 0 0 1 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0
 0 0]
